In [1]:
import IPython
import pickle, h5py, sys
import numpy as np
import networkx as nx
import pyemma as pe
import analysis_helper as ahelper
np.set_printoptions(precision=2)

/home/boltzmann/apps/anaconda/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/boltzmann/apps/anaconda/lib/python2.7/site-packages/pyemma/__init__.py:134: UserWarning: Python 2.7 usage is deprecated. Future versions of PyEMMA will not support it. Please upgrade your Python installation.
  "Please upgrade your Python installation.", category=UserWarning)


In [2]:
h = h5py.File('pre_prepped_results/west.h5')

In [3]:
a = h5py.File('pre_prepped_results/assign.h5')

In [4]:
tm = np.load('pre_prepped_results/tm.npy')
tm = tm[1:,:][:,1:]
print(tm)
# TODO: Write a plotting function to plot the matrix and show how symmetrizing changes things

[[5.28e-03 1.80e-04 9.16e-05 ... 0.00e+00 3.53e-05 2.46e-05]
 [1.85e-04 4.03e-03 4.35e-05 ... 0.00e+00 0.00e+00 9.14e-04]
 [2.47e-05 0.00e+00 2.81e-03 ... 1.75e-05 0.00e+00 0.00e+00]
 ...
 [0.00e+00 4.58e-05 2.00e-05 ... 7.62e-05 0.00e+00 2.71e-06]
 [6.27e-05 3.51e-06 0.00e+00 ... 0.00e+00 0.00e+00 0.00e+00]
 [8.30e-05 1.56e-03 0.00e+00 ... 2.43e-05 0.00e+00 1.92e-04]]


In [5]:
stm = ahelper.symmetrize(tm)
ntm = ahelper.row_normalize(stm)
print(ntm)

[[1.18e-01 4.07e-03 1.29e-03 ... 0.00e+00 1.09e-03 1.20e-03]
 [3.36e-03 7.41e-02 4.00e-04 ... 4.22e-04 3.23e-05 2.28e-02]
 [1.33e-03 4.99e-04 6.45e-02 ... 4.30e-04 0.00e+00 0.00e+00]
 ...
 [0.00e+00 4.65e-03 3.80e-03 ... 1.54e-02 0.00e+00 2.74e-03]
 [1.09e-01 3.89e-03 0.00e+00 ... 0.00e+00 0.00e+00 0.00e+00]
 [4.74e-03 1.09e-01 0.00e+00 ... 1.19e-03 0.00e+00 1.69e-02]]


In [6]:
MSM = pe.msm.MSM(ntm, reversible=True)
pcca = MSM.pcca(4)
p = pcca.coarse_grained_stationary_probability
ctm = pcca.coarse_grained_transition_matrix
print(p)

[0.1  0.17 0.34 0.39]


In [7]:
print(ctm)

[[0.62 0.01 0.2  0.17]
 [0.01 0.86 0.06 0.07]
 [0.06 0.03 0.9  0.01]
 [0.05 0.03 0.   0.92]]


In [8]:
mstable_assign = pcca.metastable_assignment

In [9]:
bin_labels = ahelper.get_bin_labels(a)

print("metastab 0")
print(bin_labels[mstable_assign.T==0].mean(axis=0))
print("metastab 1")
print(bin_labels[mstable_assign.T==1].mean(axis=0))
print("metastab 2")
print(bin_labels[mstable_assign.T==2].mean(axis=0))
print("metastab 3")
print(bin_labels[mstable_assign.T==3].mean(axis=0))

metastab 0
[4.08 3.5  0.5  0.08 0.42 0.5  0.   0.5 ]
metastab 1
[21.76 21.49  0.    1.    0.    0.    1.    0.  ]
metastab 2
[21.84  4.64  0.    0.96  0.04  0.2   0.12  0.68]
metastab 3
[ 4.92 20.72  0.2   0.08  0.72  0.    0.96  0.04]


In [ ]:
# TODO: Write a tool to plot network graphs, turn both the full matrix and the coarse matrix to graphs and plot both